# Shadows of Doubt Demo

### 0. Setup: Initialize the engine and model instance

In [1]:
# Initialize the Engine
from queue import Queue
from rtai.utils.config import YamlLoader
from rtai.llm.llm_client import LLMClient
from rtai.agent.agent_manager import AgentManager
from rtai.agent.agent import Agent
from rtai.world.world import World
from rtai.world.clock import clock

cfg = YamlLoader.load("configs/rtai_jason.yaml")
timer = clock(cfg.expand('Clock'))
event_queue = Queue()

world = World(cfg.expand('World'), event_queue)

llm_client = LLMClient()
llm_client.initialize(cfg.expand('LLMClient'))

agent_mgr = AgentManager(event_queue, cfg.expand('Agents'), client=llm_client, world=world)
agent_mgr.initialize()

mark: Agent = agent_mgr.agents['Mark Turner'] # defense attorney
sarah: Agent = agent_mgr.agents['Sarah Reynolds'] # witness
emily: Agent = agent_mgr.agents['Emily Thorton'] # defendant

/opt/homebrew/Caskroom/miniforge/base/envs/embeddings_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize the model
import guidance
from guidance import models
from guidance import gen, select

local_model_path = cfg.expand("LLMClient").get_value("local_model_path")
mistral = models.LlamaCpp(local_model_path, n_gpu_layers=-1, n_ctx=7000)

# initialize mythos as a scriptwriter
mythos = mistral + "You are a master Hollywood scriptwriter who specializes in legal dramas. Conduct each task as if you are that scriptwriter."

In [24]:
# utility functions
import json

def dump(a, filename):
    with open(filename, 'w') as f:
        json.dump(a, f)

def read_json(f_path):
    return json.loads(open(f_path).read())

# add the facts to agents' memories
def add_facts(agent, facts):
    for fact in facts:
        agent.l_mem.add_concept(fact)

def get_memory(agent):
    return list([node.summary() for node in agent.l_mem.id_to_node.values()])

def clean_dialogue(dialogue):
    cleaned = dialogue.split("\n")
    speakers = []
    utterances = []
    for line in cleaned:
        if line == '':
            continue
        else:
            split_line = line.split(":", maxsplit=1)
            try:
                speaker, utterance = split_line[0], split_line[1]
                speakers.append(speaker)
                utterances.append(utterance)
            except:
                continue
    return speakers, utterances

def show_dialogue(speakers, utterances):
    for speaker, utterance in zip(speakers, utterances):
        print(speaker+": "+utterance)

def add_dialogue(speakers, utterances):
    # add a new space for the chat
    dialogue = []
    for speaker, utterance in zip(speakers, utterances):
        dialogue.append({'speaker': speaker, 'utterance': utterance})
    return dialogue

In [4]:
# the facts of the case
sarah_facts = ["Sarah witnessed a heated argument between Emily and Richard Thornton during the social gathering.",
"She noticed Emily and Richard's strained relationship in the weeks leading up to the event.",
"Sarah is aware of financial difficulties in the Thornton marriage, with Richard contemplating a divorce.",
"During the argument, Sarah overheard Emily expressing frustration about Richard's intention to file for divorce.",
"Sarah is acquainted with other guests at the event who had their own reasons to dislike Richard Thornton."]

mark_facts = [
    "Richard had business rivals who might have had motives to harm him.",
    "There is a potential alibi for Emily during the time of the poisoning, involving a witness not yet presented in court.",
    "Inconsistencies exist in the initial police investigation, including gaps in the timeline and overlooked potential suspects.",
    "Richard was involved in a secret business deal that might have contributed to his strained relationships.",
    "Mark possesses an anonymous letter received by the Thorntons, hinting at threats from an unknown source."
]

emily_facts = [
    "Emily is aware of the severe financial struggles within her marriage to Richard.",
    "They had heated arguments about money, and Richard was considering a divorce partly due to their financial difficulties.",
    "Emily knows about the strained relationship between her and Richard leading up to the night of the incident.",
    "The tension was exacerbated by discussions about divorce, and Emily felt the emotional weight of their crumbling marriage.",
    "Emily received anonymous threatening letters in the weeks leading up to the incident.",
    "The letters hinted at harm coming to Richard and warned her about potential dangers.",
    "Emily kept this information to herself, unsure of the credibility of the threats.",
    "Emily was aware of Richard's involvement in a secret business deal that had caused friction in their personal and professional lives.",
    "The details of this deal were not fully disclosed to her, but she knew it played a role in their marriage difficulties.",
    "Emily can provide insight into her emotional state on the night of the incident.",
    "She may reveal her emotional turmoil, fears, and frustrations leading up to the events.",
    "Emily recalls an unknown person visiting their home on the night of the incident.",
    "The identity and purpose of this person remain a mystery to her."
]

In [5]:
# add the facts to the agents' memories
add_facts(sarah, sarah_facts)
add_facts(mark, mark_facts)
add_facts(emily, emily_facts)

In [6]:
to_visualize = {}

### Generate the dialogue

In [8]:
dialogue_topic = "Mark is helping Sarah prepare for taking the stand"
location = "Mark's office"

In [10]:
mark.l_mem.retriever.retrieve_context(dialogue_topic, 1)

recency score for There is a potential alibi for Emily during the time of the poisoning, involving a witness not yet presented in court. is 10.0
recency score for Mark possesses an anonymous letter received by the Thorntons, hinting at threats from an unknown source. is 10.0
recency score for Richard was involved in a secret business deal that might have contributed to his strained relationships. is 10.0
recency score for Richard had business rivals who might have had motives to harm him. is 10.0
recency score for Inconsistencies exist in the initial police investigation, including gaps in the timeline and overlooked potential suspects. is 10.0


'There is a potential alibi for Emily during the time of the poisoning, involving a witness not yet presented in court.'

In [30]:
# function to generate dialogue
@guidance
def generate_dialogue(lm, agent1, agent2, location, dialogue_topic):
    agent1_name = agent1.get_name()
    agent2_name = agent2.get_name()
    agent1_persona = agent1.get_common_set_str() # + f" {agent1_name} MUST SPEAKS LIKE A LAWYER FROM THE SOUTH"
    agent2_persona = agent2.get_common_set_str() # + f" {agent2_name} MUST SPEAK LIKE A WITNESS WHO IS ARISTOCRATIC AND BRITISH"
    agent1_ctx = agent1.l_mem.retriever.retrieve_context(dialogue_topic, 1)
    agent2_ctx = agent2.l_mem.retriever.retrieve_context(dialogue_topic)
    prompt = f'''
    Generate a short dialogue between {agent1_name} and {agent2_name} in {location} about {dialogue_topic}.

    {agent2_name}'s persona: {agent2_persona}
    {agent2_name}'s context: {agent2_ctx}
    Mark's persona: {agent1_persona}

    MAKE SURE THAT MARK ASK ABOUT {agent1_ctx}

    Example of a short dialogue:
    Mark: How are y'all doing?
    Sarah: I've been better, chap. I've been better.
    
    Here is the short dialogue:
    {gen('dialogue', temperature=1.0, max_tokens=500)}"""
    '''
    lm += prompt
    return lm

mythos += generate_dialogue(mark, emily, location, dialogue_topic)

In [12]:
# access the generated dialogue and reset the state of storyteller model
dialogue = mythos["dialogue"]
mythos.reset()

"\nMark: Emily, how are you holding up?\n\nEmily: (sighs) Mark, I've been a wreck, I can't even begin to describe the emotional turmoil I went through on that night. The fears and frustration ultimately led me here.\n\nMark: I understand, and I want to help you prepare for testifying. We'll go over the sequences of events and identify any inconsistencies that could be potential issues.\n\nEmily: Mark, do you think lies were told about that night?\n\nMark: I need to explore every possibility, Emily. Your story is crucial in understanding what happened. Have you considered the possibility of an alibi? There might be a witness who could've potentially been present during the time of the poisoning.\n\nEmily: What ... a witness? Mark, I have no recollection of anyone being there.\n\nMark: Just hear me out. What about that letter you received? The one with threats from an unknown source? Do you remember anything about when that letter was mailed or who might have sent it?\n\nEmily: I think i

In [26]:
speakers, utterances = clean_dialogue(dialogue)
to_visualize['mark_emily_dialogue'] = add_dialogue(speakers=speakers, utterances=utterances)

In [29]:
to_visualize

{'mark_initial_memories': ['Richard had business rivals who might have had motives to harm him.',
  'There is a potential alibi for Emily during the time of the poisoning, involving a witness not yet presented in court.',
  'Inconsistencies exist in the initial police investigation, including gaps in the timeline and overlooked potential suspects.',
  'Richard was involved in a secret business deal that might have contributed to his strained relationships.',
  'Mark possesses an anonymous letter received by the Thorntons, hinting at threats from an unknown source.'],
 'mark_emily_dialogue': [{'speaker': 'Mark',
   'utterance': ' Emily, how are you holding up?'},
  {'speaker': 'Emily',
   'utterance': " (sighs) Mark, I've been a wreck, I can't even begin to describe the emotional turmoil I went through on that night. The fears and frustration ultimately led me here."},
  {'speaker': 'Mark',
   'utterance': " I understand, and I want to help you prepare for testifying. We'll go over the 

In [ ]:
mark.l_mem.retriever.retrieve_context(dialogue_topic)

In [ ]:
mark.l_mem.id_to_node

In [ ]:
dialogue

In [ ]:
mark_ctx = mark.l_mem.retriever.retrieve_context(dialogue_topic)
mark_ctx

### Reflect on the dialogue

In [ ]:
from guidance import select

@guidance
def reflect(lm, agent1, agent2, dialogue):
    agent1_name = agent1.get_name()
    agent2_name = agent2.get_name()
    agent1_ctx = agent1.l_mem.retriever.retrieve_context(dialogue)
    lm.set("agent1_ctx", agent1_ctx)
    lm += f'''Before chattting with {agent2_name}, {agent1_name} knew {agent1_ctx}.

    Given this dialogue, what are the top 3 NEW takeaways for {agent1_name} from this dialogue?

    Here is the dialogue: {dialogue}

    Here are the takeaways:
    {gen(name="takeaways", temperature=0.8, max_tokens=300)}'''

    # next action
    lm += f'''
    Based off those three takwaways what should {agent1_name} do next?

    {agent1_name}'s options are to THEORIZE: take time to theorize a timeline about what happened, CHAT: go chat with another witness to gather more information
    {select(name="do_next", options=["THEORIZE", "CHAT"])} because {gen(name="reason", temperature=0.8, max_tokens=300)}'''
    if lm["do_next"] == "THEORIZE":
        lm += f'''Q: What is {agent1_name}'s current theory about what happened?
        A: {gen(name="theory", temperature=0.8, max_tokens=300)}.'''
    if lm["do_next"] == "CHAT":
        lm += f'''Q: Who should {agent1_name} chat with next to learn more information about the case? Prioritize witnesses who have not been interviewed yet.

        Think step by steps
        {gen(name="reasoning", temperature=0.8, max_tokens=300)}

        I will now select with who to chat next with:
        Who to chat next with: {select(name="next_chat", options=["Emily Thorton", "Emily Thorton"])}. Topic of the next chat: {gen(name="topic", stop=".", temperature=0.8, max_tokens=100)}.'''
    return lm

prompt = f'''Who should Mark chat with next? {select(name="next_chat", options=["Emily Thorton", "Sarah"])}'''
mythos_reasoning = mistral + "You are analyzing what Mark Turner learned from a chat with Sarah."
mythos_reasoning += reflect(agent1=mark, agent2=sarah, dialogue=dialogue)
mythos_reasoning += prompt

In [ ]:
mythos_reasoning["takeaways"]

In [ ]:
mythos_reasoning.reset()

In [ ]:
mythos_reasoning["topic"]

In [ ]:
# Now we add the takeaways from the dialogue to Mark's memory
takeaways = mythos_reasoning["takeaways"].split("\n")
out = []
# clean the takeaways
for takeaway in takeaways:
    if takeaway == '':
        continue
    else:
        cleaned_takeaway = takeaway.split(".", maxsplit=1)[1]
        print(cleaned_takeaway)
        out.append(cleaned_takeaway)
        mark.l_mem.add_concept(cleaned_takeaway)

In [ ]:
dump(out, "mark_reflection1.json")

In [ ]:
a = list([node.summary() for node in mark.l_mem.id_to_node.values()])

import json

with open('mark_facts_after.json', 'w') as f:
    json.dump(a, f)

In [ ]:
json.loads(open('mark_facts_after.json').read())

In [ ]:
mark.l_mem.retriever.retrieve_context("Emily")

In [ ]:
chat_next = mythos_reasoning["next_chat"]
about = mythos_reasoning["topic"]
mythos_reasoning.reset()

In [ ]:
# do next action
next_action = mythos_reasoning["do_next"]
who = mythos_reasoning["next_chat"]
f"Mark's next action is to {next_action} with {who}"

In [ ]:
mythos.reset()

In [ ]:
mythos += generate_dialogue(mark, emily, location, dialogue_topic=about)

In [ ]:
# access the generated dialogue and reset the state of storyteller model
dialogue = mythos["dialogue"]
mythos.reset()

In [ ]:
import pandas as pd

speakers, utterances = clean_dialogue(dialogue)
df2 = pd.DataFrame({'speaker': speakers, 'utterance': utterances})
df2.to_csv('dialogues/dialogue6.csv', index=False)

In [ ]:
# Now we summarize
df3 = pd.read_csv('dialogues/dialogue5.csv')
df4 = pd.read_csv('dialogues/dialogue6.csv')

In [ ]:
def string_rep(df):
    concatenated_string = ', '.join(df3.apply(lambda row: f"{row['speaker']}:{row['utterance']}", axis=1))
    return concatenated_string

In [ ]:
prompt = f'''

Here are character summaries for Mark, Emily, and Sarah:
Mark Character Summary: {mark.get_common_set_str()}
Emily Character Summary: {emily.get_common_set_str()}
Sarah Character Summary: {sarah.get_common_set_str()}

Here is the first conversation between Mark and Sarah:
{string_rep(df3)}

Here is the second conversation between Mark and Emily:
{string_rep(df4)}

Now, create a detective fiction vignette from Mark's persepctive (SPEAK IN THIRD PERSON) for a young adult audience that incorporates each character's personality and quotes from the two conversations between Mark and Sarah and Mark and Emily. Make it dramatic and interesting to read. Keep each line SHORT AND SNAPPY. Incorporate metaphors and symbolism.
{gen(name="story", temperature=1.0, max_tokens=1000)}
'''

In [ ]:
fiction = mistral + "You are a seasoned detective fiction writer. You incorporate inspiration tastefully and in an entertaining way."
mythos += prompt

In [ ]:
story_line = []
mythos["story"].split("\n")
for line in mythos["story"].split("\n"):
    if line == '':
        continue
    else:
        story_line.append(line)
story_df = pd.DataFrame(story_line, columns=['story'])
story_df.to_csv('dialogues/story.csv', index=False)